Used the Car Accident Detection and Prediction (CADP) dataset from Carnegie Mellon University and the University of Tokyo.
From the manually annotated part of the dataset, we extracted the original CCTV videos.
Using the provided annotation file, we identified the time intervals where accidents occur.
For each video, we selected one frame per second within those accident intervals and saved them as accident images.
All extracted frames were saved into a structured folder for further model training and analysis.


In [4]:
%pip install opencv-python
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 22.0 MB/s  0:00:01 eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.9/37.9 MB 27.1 MB/s  0:00:01 eta 0:00:01
  Created wheel for numpy: filename=numpy-2.2.6-cp314-cp314-macosx_15_0_arm64.whl size=5145560 sha256=97a0de928c500c97606d5851b2b321fe7a5fff96d85aa93ea3c86eecaa56bdbc
  Stored in directory: /Users/ddsharafiev/Library/Caches/pip/wheels/83/be/bb/986b1bdc6955e697e6c56b03ef4aa1d67eb2c9fbeaf8e2804f
Successfully built numpy
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.4
    Uninstalling numpy-2.3.4:
      Successfully uninstalled numpy-2.3.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [opencv-python]0m [opencv-python]
Note: you may need to restart the kernel to use updated packages.


In [5]:
import json
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from pathlib import Path
import shutil

In [7]:
videos_dir = "raw_data/manual/videos"
annotations_path = "raw_data/manual/annotations.json"

video_files = {os.path.basename(f) for f in os.listdir(videos_dir) if f.endswith(".mp4")}

with open(annotations_path, "r") as f:
    annotations = json.load(f)
json_keys = set(annotations.keys())

intersection = video_files.intersection(json_keys)
missing_in_videos = json_keys - video_files
missing_in_json = video_files - json_keys

print(f"Total video files: {len(video_files)}")
print(f"Total entries in JSON: {len(json_keys)}")
print(f"Matched files: {len(intersection)}\n")


Total video files: 3
Total entries in JSON: 226
Matched files: 3



In [9]:
import os
import json
import cv2
from pathlib import Path
import math

videos_dir = Path("raw_data/manual/videos")
annotations_path = Path("raw_data/manual/annotations.json")
output_root = Path("dataset_accident_frames")

(output_root / "Accident").mkdir(parents=True, exist_ok=True)

with open(annotations_path, "r") as f:
    annotations = json.load(f)

for video_name in os.listdir(videos_dir):
    if not video_name.endswith(".mp4"):
        continue

    video_path = videos_dir / video_name
    ann = annotations.get(video_name, [])
    if not ann or not ann[0].get("keyframes"):
        continue  

    keyframes = ann[0]["keyframes"]
    starts = [k["frame"] for k in keyframes if k.get("state") == "Start"]
    ends = [k["frame"] for k in keyframes if k.get("state") == "End"]

    if not starts or not ends:
        continue  

    start_sec = float(starts[0])
    end_sec = float(ends[0])

    cap = cv2.VideoCapture(str(video_path))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    
    for sec in range(math.floor(start_sec), math.ceil(end_sec)):
        frame_idx = int(sec * fps)
        if frame_idx >= total_frames:
            break

        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        success, frame = cap.read()
        if success:
            out_name = f"{Path(video_name).stem}_{sec:04d}s.jpg"
            out_path = output_root / "Accident" / out_name
            cv2.imwrite(str(out_path), frame)

    cap.release()
